In [1]:
### IMPORTS

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense, Dropout, LeakyReLU
# from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras import backend
from keras import optimizers
import keras
import folium
from folium.plugins import AntPath, MousePosition
from sklearn.utils import shuffle, resample
from keras.models import Model, load_model, Sequential
from keras.layers import Dense, Dropout, LSTM, Input, RepeatVector, GRU, Bidirectional
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import os
from datetime import datetime
import random
import math
from tqdm import tqdm
from datetime import datetime
from math import radians, cos, sin, asin, sqrt
from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms
from sklearn.metrics import silhouette_score
from scipy.stats import gaussian_kde
import seaborn as sns
from sklearn.mixture import GaussianMixture
from scipy.spatial import ConvexHull
from shapely.geometry import Polygon
from sklearn.metrics import mean_squared_error 


In [14]:
### Making list of all models
a = []
for i in os.listdir(r'D:\Python\master thesis\master-thesis\all_models'):
    if i.split('.')[1]=='h5':
        # print(i)
        a.append([i.split('-cells-')[0],i.split('-cells-')[1].split('-epochs-')[0],i.split('-batch-')[1].split('-test-')[0]])


In [20]:
### DATASET LOAD

df = pd.read_csv(r"D:\Python\master thesis\data\final_dataset.csv")
df = df.drop(columns=['Unnamed: 0'])
df['# Timestamp'] = pd.to_datetime(df['# Timestamp'])
df = df.sort_values(by=['MMSI', '# Timestamp'])
# # # Display the first few rows of the dataset to understand its structure

In [21]:
def diff_to_coord(y,X):
    a = np.zeros_like(y)
    for j in tqdm(range(a.shape[0])):
        for i in range(a.shape[1]):
            if i == 0:
                a[j,i,0] = X[j,-1,0] - y[j,i,0]
                a[j,i,1] = X[j,-1,1] - y[j,i,1]
            else:
                a[j,i,0] = a[j,i-1,0] - y[j,i,0]
                a[j,i,1] = a[j,i-1,1] - y[j,i,1]
    return a

In [22]:
seq_input_length = 30
seq_output_length = 20
seq_length = seq_input_length+seq_output_length
window_size = seq_length // 2

seq = []
for i in tqdm(range(0, len(df) - seq_length + 1, window_size)):
    temp = df[i: i + seq_length].values
    # check if all sequences hold only one MMSI (first == last)
    if(temp[0,0] == temp[-1,0]):
        seq.append(temp)
seq = np.dstack(seq)
seq = np.rollaxis(seq,-1)
seq = shuffle(seq,random_state=42)
seq.shape

100%|██████████| 119585/119585 [00:25<00:00, 4746.36it/s]


(116864, 50, 12)

In [23]:
bad_sequences = []
for index,i in tqdm(enumerate(seq)):
    # print(index)
    if 0 in i[:,4] or (i[:,6] > 2).sum() > 0 or (i[:,6] < 1).sum() > 0 or 0 in (np.append(np.atleast_2d(i[:,7:9].sum(axis=1)).T,i[:,7:8],axis=1)).sum(axis=1) or np.isnan((i[:,2:].astype(np.float32))).sum() > 0 or (i[:,11] <= 0).sum() > 0 or (i[:,11] > 800).sum() > 0:
        bad_sequences.append(index)
new_seq = np.delete(seq,bad_sequences,axis=0)
new = new_seq.reshape((new_seq.shape[0]*new_seq.shape[1],new_seq.shape[2]))

116864it [00:05, 20064.52it/s]


In [26]:
df = pd.DataFrame(new,columns=['MMSI','# Timestamp','Latitude', 'Longitude' ,'SOG','Heading','DateDiff','Lat_speed','Long_speed','Lat_lag','Long_lag','haversine_distance'])

In [27]:
df['d_Lat'] =  df['Lat_lag'] - df['Latitude']
df['d_Long'] = df['Long_lag'] - df['Longitude']

In [28]:
### NORMALIZING VALUES

small_df = df[['MMSI','# Timestamp']]
df = df.drop(columns=['MMSI','# Timestamp'])
features=[i for i in df.columns]
scaler = MinMaxScaler()
 
small_df[features] = scaler.fit_transform(df[features])
df = small_df

In [29]:
df_numpy = df.to_numpy()
seq = df_numpy.reshape((new_seq.shape[0],new_seq.shape[1],df.shape[1]))

In [97]:
### DIVIDING INTO FEATURE (X) AND TARGET VARIABLES (y)

X = seq[:,0:seq_input_length,:]
y = seq[:,seq_input_length:seq_length,:]

In [98]:
### SPLITTING DATA INTO TRAIN,TEST,VALIDATION (70%,15%,15%)
def split_prep(X,y,random_state):
    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15,random_state=42)
    # X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.176,random_state=random_state)   # 70 : 15 : 15
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=42)
    X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5,random_state=42)   # 70 : 15 : 15

    X_train_primary = X_train
    X_test_primary = X_test
    X_val_primary = X_val
    y_train_primary = y_train
    y_test_primary = y_test
    y_val_primary = y_val


    X_train = X_train[:,:,2:]
    X_test = X_test[:,:,2:]
    X_val = X_val[:,:,2:]

    # y_train = y_train[:,:,2:4] # drop all features except lat and long
    # y_val = y_val[:,:,2:4] # drop all features except lat and long
    # y_test = y_test[:,:,2:4] # drop all features except lat and long

    y_train = y_train[:,:,-2:] # drop all features except lat and long
    y_val = y_val[:,:,-2:] # drop all features except lat and long
    y_test = y_test[:,:,-2:] # drop all features except lat and long

    X_train=X_train.astype('float32')
    X_test=X_test.astype('float32')
    X_val=X_val.astype('float32')
    y_train=y_train.astype('float32')
    y_test=y_test.astype('float32')
    y_val=y_val.astype('float32')

    y_train_flat = np.copy(y_train)
    y_val_flat = np.copy(y_val)
    y_test_flat = np.copy(y_test)
    y_train_flat = y_train_flat.reshape(y_train_flat.shape[0], y_train_flat.shape[1]*y_train_flat.shape[2])
    y_val_flat = y_val_flat.reshape(y_val_flat.shape[0], y_val_flat.shape[1]*y_val_flat.shape[2])
    y_test_flat = y_test_flat.reshape(y_test_flat.shape[0], y_test_flat.shape[1]*y_test_flat.shape[2])
    
    return X_train, X_test, X_val, y_train_flat, y_test_flat, y_val_flat, y_train, y_test, y_val

In [103]:
X_train, X_test, X_val, y_train_flat, y_test_flat, y_val_flat, y_train, y_test, y_val = split_prep(X,y,2)

In [70]:
def haversine(lat1, lon1, lat2, lon2, radius=6371)
    # Convert degrees to radians
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    
    # Differences
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    # Haversine formula
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return radius * c

In [104]:
y_test = np.concatenate((np.zeros((y_test.shape[0], y_test.shape[1], X_test.shape[2] - y_test.shape[2])),y_test),axis=2)
y_test = scaler.inverse_transform(y_test.reshape(y_test.shape[0]*y_test.shape[1],y_test.shape[2]))[:,-2:]
y_test = y_test.reshape((int(y_test.shape[0]/seq_output_length),seq_output_length,2))

In [72]:
### Predicting with all models
model_names = []
predictions = []
for model_name in os.listdir('D:\Python\master thesis\master-thesis\\all_models'):
    if '.h5' in model_name:
        print(model_name)
        model = load_model(f'D:\Python\master thesis\master-thesis\\all_models\{model_name}')
        y_pred_test = model.predict(X_test)

        y_pred_test= y_pred_test.reshape(y_pred_test.shape[0],y_test.shape[1],y_test.shape[2])

        y_pred_test = np.concatenate((np.zeros((y_pred_test.shape[0], y_pred_test.shape[1], X_test.shape[2] - y_test.shape[2])),y_pred_test),axis=2)
        y_pred_test = scaler.inverse_transform(y_pred_test.reshape(y_pred_test.shape[0]*y_pred_test.shape[1],y_pred_test.shape[2]))[:,-2:]
        y_pred_test = y_pred_test.reshape((int(y_pred_test.shape[0]/seq_output_length),seq_output_length,2))
        
        model_names.append(model_name)
        predictions.append(y_pred_test)
        
        # break
        


Bi_LSTM-cells-100-epochs-100-batch-128-test-0-datetime-2024_12_18-20_35_50.h5
329/329 [==============================] - 2s 5ms/step
Bi_LSTM-cells-100-epochs-100-batch-16-test-0-datetime-2024_12_22-13_21_52.h5
329/329 [==============================] - 2s 5ms/step
Bi_LSTM-cells-100-epochs-100-batch-32-test-0-datetime-2024_12_23-11_08_29.h5
329/329 [==============================] - 2s 5ms/step
Bi_LSTM-cells-100-epochs-100-batch-64-test-0-datetime-2024_12_18-21_08_37.h5
329/329 [==============================] - 2s 5ms/step
Bi_LSTM-cells-150-epochs-100-batch-128-test-0-datetime-2024_12_18-22_15_26.h5
329/329 [==============================] - 2s 7ms/step
Bi_LSTM-cells-150-epochs-100-batch-16-test-0-datetime-2024_12_22-17_35_28.h5
329/329 [==============================] - 2s 7ms/step
Bi_LSTM-cells-150-epochs-100-batch-32-test-0-datetime-2024_12_23-13_17_01.h5
329/329 [==============================] - 2s 6ms/step
Bi_LSTM-cells-150-epochs-100-batch-64-test-0-datetime-2024_12_23-16_23_17.

In [105]:
X_test = scaler.inverse_transform(X_test.reshape(X_test.shape[0]*X_test.shape[1],X_test.shape[2]))
X_test = X_test.reshape((int(X_test.shape[0]/seq_input_length),seq_input_length,X_train.shape[2]))
y_test = diff_to_coord(y_test,X_test)
predictions_new = []
for y_pred in predictions:
    y_pred_test = diff_to_coord(y_pred,X_test)
    predictions_new.append(y_pred_test)
predictions = predictions_new

100%|██████████| 10505/10505 [00:00<00:00, 54423.51it/s]


In [79]:
### MAHE
def mean_absolute_haversine_error(y_true, y_pred, radius=6371):
    # Extract latitudes and longitudes
    lat_true, lon_true = y_true[:, 0], y_true[:, 1]
    lat_pred, lon_pred = y_pred[:, 0], y_pred[:, 1]
    
    # Calculate haversine distances
    distances = haversine(lat_true, lon_true, lat_pred, lon_pred, radius)
    
    # Return mean absolute error
    return np.mean(np.abs(distances))

# Example Usage
y_true = np.array([[40.7128, -74.0060], [34.0522, -118.2437]])  # True coordinates (lat, lon)
y_pred = np.array([[40.7127, -74.0059], [34.0521, -118.2436]])  # Predicted coordinates (lat, lon)

mahe = mean_absolute_haversine_error(y_true, y_pred)
print(f"Mean Absolute Haversine Error: {mahe} km")

Mean Absolute Haversine Error: 0.014196518661889448 km


In [ ]:
new = []
for i,prediction in tqdm(enumerate(predictions)):
    
    mahe = mean_absolute_haversine_error(y_test.reshape(y_test.shape[0]*y_test.shape[1],y_test.shape[2]),prediction.reshape(prediction.shape[0]*prediction.shape[1],prediction.shape[2]))
    new.append([model_names[i],round(mahe*1000,2)])
    print(mahe)

0.2233430624935147
0.20514673957221152
0.21488851546333998
0.2231698585508794
0.23924249585995616
0.2000548501356808
0.21015316876559537
0.2127124182017258
0.21612413071223716
0.19250640832626667
0.2056308719440531
0.21665237935747467
0.21298856595891189
0.20826774987588748
0.20228553865040985
0.21384148739107592
0.23319035781049746
0.21714226571150402
0.4685379089579962
0.23782899078612782
0.26615025936345144
0.21193709610803
0.20713834688447913
0.22940940924133393
0.273917453973571
0.20538726881527394
0.21023577443616465
0.2312148078412546
0.25220715910491087
0.19683645265839259
0.21234359497828656
0.24523687593599403
0.2477661566022576
0.19462012410455412
0.20590698894578977
0.2509452516852115
0.6852700084128992
0.21808405421343205
0.24337375561916674
0.2385935400534298
0.23563447715301628
0.2182800480965827
0.20757907789632846
0.22985037791213392
0.22798450338306187
0.2129459521583177
0.22072729798275506
0.22360692169151225
0.221811200403134
0.21297681070098334
0.2156737688792549
0

In [140]:
error_rates = pd.read_csv('new_error_rates.csv')

In [141]:
### Adding MAHE
error_rates_new = []
for i in error_rates.iterrows():
    # print(i[1][1])
    error_rates_new.append([i[1][1].split('-cells-')[0],i[1][1].split('-cells-')[1].split('-epochs-')[0],i[1][1].split('-batch-')[1].split('-test-')[0],i[1][2]])

In [143]:
error_rates_new

[['Bi_LSTM', '100', '128', 223.34],
 ['Bi_LSTM', '100', '16', 205.15],
 ['Bi_LSTM', '100', '32', 214.89],
 ['Bi_LSTM', '100', '64', 223.17],
 ['Bi_LSTM', '150', '128', 239.24],
 ['Bi_LSTM', '150', '16', 200.05],
 ['Bi_LSTM', '150', '32', 210.15],
 ['Bi_LSTM', '150', '64', 212.71],
 ['Bi_LSTM', '200', '128', 216.12],
 ['Bi_LSTM', '200', '16', 192.51],
 ['Bi_LSTM', '200', '32', 205.63],
 ['Bi_LSTM', '200', '64', 216.65],
 ['Bi_LSTM', '250', '128', 212.99],
 ['Bi_LSTM', '250', '16', 208.27],
 ['Bi_LSTM', '250', '32', 202.29],
 ['Bi_LSTM', '250', '64', 213.84],
 ['Bi_LSTM', '50', '128', 233.19],
 ['Bi_LSTM', '50', '16', 217.14],
 ['Bi_LSTM', '50', '32', 468.54],
 ['Bi_LSTM', '50', '64', 237.83],
 ['GRU', '100', '128', 266.15],
 ['GRU', '100', '16', 211.94],
 ['GRU', '100', '32', 207.14],
 ['GRU', '100', '64', 229.41],
 ['GRU', '150', '128', 273.92],
 ['GRU', '150', '16', 205.39],
 ['GRU', '150', '32', 210.24],
 ['GRU', '150', '64', 231.21],
 ['GRU', '200', '128', 252.21],
 ['GRU', '200', '

In [144]:
pd.DataFrame(error_rates_new).to_csv('newnew_error_rates_mahe.csv')